In [ ]:
import os
import json
from googletrans import Translator
from pathlib import Path

def create_caption_files(json_dir, image_dir, txt_dir):
    """
    JSON 라벨 파일에서 한국어 캡션을 읽어 영어로 번역한 후,
    지정된 별도의 txt 디렉토리에.txt 파일로 저장합니다.

    Args:
        json_dir (str): JSON 라벨 파일이 있는 디렉토리 경로.
        image_dir (str): 이미지 파일이 있는 디렉토리 경로.
        txt_dir (str): 생성될.txt 캡션 파일을 저장할 디렉토리 경로.
    """
    # 디렉토리 경로를 Path 객체로 변환하여 OS 호환성 확보
    json_path = Path(json_dir)
    image_path = Path(image_dir)
    txt_path = Path(txt_dir)

    # 입력 디렉토리가 존재하는지 확인
    if not json_path.is_dir():
        print(f"오류: JSON 디렉토리 '{json_dir}'를 찾을 수 없습니다.")
        return
    if not image_path.is_dir():
        print(f"오류: 이미지 디렉토리 '{image_dir}'를 찾을 수 없습니다.")
        return

    # 출력 디렉토리 생성 (이미 존재하면 아무 작업도 하지 않음)
    txt_path.mkdir(parents=True, exist_ok=True)

    # 구글 번역기 초기화
    translator = Translator()

    # 처리된 파일과 건너뛴 파일 카운트
    processed_count = 0
    skipped_count = 0

    print(f"캡션 파일 생성을 시작합니다... (저장 위치: '{txt_dir}')")

    # json 디렉토리 내의 모든.json 파일 순회
    for label_file in json_path.glob('*.json'):
        try:
            with open(label_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # JSON 데이터에서 이미지 파일명과 한국어 캡션 추출
            image_filename = data.get("images", {}).get("identifier")
            korean_caption = data.get("images", {}).get("caption_kr")

            if not image_filename or not korean_caption:
                print(f"경고: '{label_file.name}' 파일에 'identifier' 또는 'caption_kr' 필드가 없습니다. 건너뜁니다.")
                skipped_count += 1
                continue

            # 대응하는 이미지 파일의 전체 경로 생성
            target_image_path = image_path / image_filename

            # 이미지 파일이 실제로 존재하는지 확인 (오류 방지)
            if not target_image_path.is_file():
                print(f"알림: '{image_filename}'에 해당하는 이미지 파일이 없습니다. 건너뜁니다.")
                skipped_count += 1
                continue

            # 한국어 캡션을 영어로 번역
            try:
                translated = translator.translate(korean_caption, src='ko', dest='en')
                english_caption = translated.text
            except Exception as e:
                print(f"번역 오류 발생 ('{label_file.name}'): {e}. 건너뜁니다.")
                skipped_count += 1
                continue

            #.txt 캡션 파일 경로 생성 (예: image.jpg -> image.txt)
            # 저장 위치를 txt_path로 지정
            caption_filename = txt_path / Path(image_filename).with_suffix('.txt')

            # 번역된 캡션을.txt 파일로 저장
            with open(caption_filename, 'w', encoding='utf-8') as caption_f:
                caption_f.write(english_caption)

            print(f"생성 완료: '{caption_filename}'")
            processed_count += 1

        except json.JSONDecodeError:
            print(f"오류: '{label_file.name}' 파일이 올바른 JSON 형식이 아닙니다. 건너뜁니다.")
            skipped_count += 1
        except Exception as e:
            print(f"알 수 없는 오류 발생 ('{label_file.name}'): {e}. 건너뜁니다.")
            skipped_count += 1
            
    print("\n--- 작업 완료 ---")
    print(f"총 생성된 캡션 파일: {processed_count}개")
    print(f"총 건너뛴 파일: {skipped_count}개")


In [ ]:
    # 경로 설정
    json_directory = './minhwa_dataset/json/'
    image_directory = './minhwa_dataset/jpg/'
    txt_output_directory = './minhwa_dataset/txt/' # 캡션 파일을 저장할 폴더

    # 함수 실행
    create_caption_files(json_directory, image_directory, txt_output_directory)